In [1]:
##Robs summariser of RC9 Additional Scenario Regional Contributor tables
import pandas as pd
import numpy as np
#import libarchive
#import tempfile
import os
#from functools import reduce
import datetime

#mainPath = 'P:\projects\RC9_ResultsSets\Tester'
mainPath = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth'

#outPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
outPath = 'P:\projects\RC10_ResultsSets\GrzGullyAvFinePercent'
#summariesOutFolder = 'Summaries'


parameterFileName = 'ParameterTable.csv'
areaFileName = 'fuAreasTable.csv'

#theTempDir = tempfile.gettempdir()
#theCurrentDir = os.getcwd()

#print("Current dir is: " + theCurrentDir)

#pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC9_RC2017\RC9_ResultsSets_PointOfTruth\constituentsToGroups.csv'

regList = ['BM','BU', 'CY', 'FI','MW','WT']
#regList = ['BM','MW','WT']
#regList = ['BU']


filterColName = 'CONSTITUENT'
constitsList = ['Sediment - Fine']

fuColName = 'ELEMENT'
fuList = ['Grazing Forested', 'Grazing Open']

#modelColName = 'MODEL'
#modelNames = ['In Stream Fine Sediment Model - SedNet']

paramColName = 'PARAMETER'
paramNames = ['Gully Clay + Silt Percentage']

#scenarioToFolderDict = {baselineScenarioName:baseFolderEnd, changeScenarioName:changeFolderEnd, predevScenarioName:predevFolderEnd}


In [2]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [3]:
def getPathInfo(regionIDString, fileName):
    fileIn = mainPath + '\\' + regionIDString + '\\Model_Outputs\\' + regionIDString + '_BASE_RC10\\' + fileName
    #return fileIn, fileOut
    return fileIn


In [4]:
def processParameterFiltering(regionIDString):
    
    #theFileIn, theFileOut = getPathInfo(regionIDString)
    theFileIn = getPathInfo(regionIDString, parameterFileName)
    theAreasFile = getPathInfo(regionIDString, areaFileName)
    
    print("Processing " + theFileIn)
    
    #This was for reading out of zip fiels
    #with libarchive.file_reader(theFileIn) as e:
    #    for entry in e:
    #        if(regContributorFileName in entry.pathname):
    #            print(entry)
    #            with open(theTempDir + "\\" + regContributorFileName, 'wb') as f:
    #                for block in entry.get_blocks():
    #                    f.write(block)
    #                break
    
    rawPT = pd.read_csv(theFileIn)
    
    filtered1 = rawPT.loc[rawPT[filterColName].isin(constitsList)]
    filtered2 = filtered1.loc[filtered1[fuColName].isin(fuList)]
    filtered3 = filtered2.loc[filtered1[paramColName].isin(paramNames)]
    #The parameter value will be an 'object' at the moment
    filtered3['VALUE'] = pd.to_numeric(filtered3['VALUE'], downcast="float")
    
    rawAreas = pd.read_csv(theAreasFile)
    filteredAreas = rawAreas.loc[rawAreas['FU'].isin(fuList)]
    filteredAreas = filteredAreas.rename(columns={'Area':'FUArea'})
    catchAreas = filteredAreas.groupby(['Catchment']).agg({'FUArea':'sum'}).reset_index()
    catchAreas = catchAreas.rename(columns={'FUArea':'CATCHArea'})
    mergedAreas = pd.merge(filteredAreas,catchAreas, how='inner', left_on=['Catchment'], right_on = ['Catchment'])
    
    mergedAreas['FUProp'] = mergedAreas['FUArea'] / mergedAreas['CATCHArea']
    mergedAreas['FUProp'].fillna(0.0, inplace=True)
    
    joinedParams = pd.merge(filtered3, mergedAreas, how='inner', left_on=['CATCHMENT','ELEMENT'], right_on = ['Catchment','FU'])
    
    #print(joinedParams.dtypes)
    joinedParams['WeightedVal'] = joinedParams['FUProp'] * joinedParams['VALUE']
    
    finalSum = joinedParams.groupby(['Catchment']).agg({'WeightedVal':'sum'}).reset_index()
    
    theFileOut = outPath + '\\' + regionIDString + '_GrzAverageGullyFinePercent.csv'
    print("Processing " + theFileOut)
    finalSum.to_csv(theFileOut, index=False)
    
    return

    

In [5]:
#List out the processes

for regID in regList:
    processParameterFiltering(regID)

print("Finished first summary " + str(datetime.datetime.now()))

Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BM\Model_Outputs\BM_BASE_RC10\ParameterTable.csv


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing P:\projects\RC10_ResultsSets\GrzGullyAvFinePercent\BM_GrzAverageGullyFinePercent.csv
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BU\Model_Outputs\BU_BASE_RC10\ParameterTable.csv
Processing P:\projects\RC10_ResultsSets\GrzGullyAvFinePercent\BU_GrzAverageGullyFinePercent.csv
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\CY\Model_Outputs\CY_BASE_RC10\ParameterTable.csv
Processing P:\projects\RC10_ResultsSets\GrzGullyAvFinePercent\CY_GrzAverageGullyFinePercent.csv
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\FI\Model_Outputs\FI_BASE_RC10\ParameterTable.csv
Processing P:\projects\RC10_ResultsSets\GrzGullyAvFinePercent\FI_GrzAverageGullyFinePercent.csv
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\MW\Model_Outputs\MW_BASE_RC10\ParameterTable.csv
Processing P:\projects\RC10_ResultsSets\GrzGullyAvFinePercent\MW_GrzAverageGullyFinePercent.csv
Processi